In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

def get_ace_editor_value(driver, ace_element):
    """Ace Editor에서 값을 가져오는 함수"""
    try:
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script("return typeof ace !== 'undefined' && ace.edit(arguments[0]) !== null", ace_element)
        )
        value = driver.execute_script("return ace.edit(arguments[0]).getValue();", ace_element).strip()
        return value
    except Exception as e:
        print(f"Ace Editor 값 가져오기 실패: {e}")
        return None

def extract_column_data(driver):
    """컬럼 데이터 추출 함수"""
    data_list = []
    
    # 모든 컬럼 블록 찾기
    columns = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder_1rntr_40"))
    )
    
    for column in columns:
        try:
            # 컬럼 명 추출
            col_name_label = column.find_element(By.XPATH, ".//div[contains(@class, 'inline field')][1]/label")
            col_name_value = column.find_element(By.XPATH, ".//div[contains(@class, 'inline field')][1]/p")
            col_name = col_name_value.text if col_name_value else "N/A"

            # 데이터 타입 추출
            data_type_label = column.find_element(By.XPATH, ".//div[contains(@class, 'inline field')][2]/label")
            data_type_value = column.find_element(By.XPATH, ".//div[contains(@class, 'inline field')][2]/p")
            data_type = data_type_value.text if data_type_value else "N/A"

            # 값 추출
            value_field = column.find_elements(By.XPATH, ".//div[contains(@class, 'inline field')][3]/*[2]")
            if value_field:
                # Ace Editor가 있는 경우
                try:
                    ace_editor = value_field[0].find_element(By.CLASS_NAME, "ace_editor")
                    value = get_ace_editor_value(driver, ace_editor)
                    if value is None:
                        value = "N/A"
                except NoSuchElementException:
                    # 일반 <p> 태그 또는 placeholder 처리
                    try:
                        p_tag = value_field[0].find_element(By.TAG_NAME, "p")
                        value = p_tag.text if p_tag else "N/A"
                    except NoSuchElementException:
                        # Placeholder가 있으면 "N/A"로 처리
                        value = "N/A" if "ui placeholder" in value_field[0].get_attribute("class") else "Unknown"

            # 결과 리스트에 추가
            data_list.append({
                "컬럼 명": col_name,
                "데이터 타입": data_type,
                "값": value
            })
            print(f"추출 완료: {col_name} | {data_type} | {value}")

        except NoSuchElementException as e:
            print(f"요소 찾기 실패: {e}")
            continue
        except TimeoutException:
            print("시간 초과로 요소를 찾지 못함")
            continue

    return data_list

# 실행 예시
def main():
    # Selenium WebDriver 설정 (예: Chrome)
    from selenium import webdriver
    driver = webdriver.Chrome()  # ChromeDriver 경로가 PATH에 설정되어 있어야 함
    driver.get("your_url_here")  # 실제 웹페이지 URL 입력

    try:
        # 데이터 추출
        result = extract_column_data(driver)
        
        # 결과 출력
        for item in result:
            print(f"컬럼 명: {item['컬럼 명']}, 데이터 타입: {item['데이터 타입']}, 값: {item['값']}")
            
    finally:
        time.sleep(2)  # 브라우저 종료 전 대기
        driver.quit()

if __name__ == "__main__":
    main()

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x00007FF712A66EE5+28773]
	(No symbol) [0x00007FF7129D25D0]
	(No symbol) [0x00007FF712868DDC]
	(No symbol) [0x00007FF712855B67]
	(No symbol) [0x00007FF712853F62]
	(No symbol) [0x00007FF7128545CC]
	(No symbol) [0x00007FF71286CDFA]
	(No symbol) [0x00007FF7129105AE]
	(No symbol) [0x00007FF7128E724A]
	(No symbol) [0x00007FF71290F6F3]
	(No symbol) [0x00007FF7128E7023]
	(No symbol) [0x00007FF7128AFF5E]
	(No symbol) [0x00007FF7128B11E3]
	GetHandleVerifier [0x00007FF712DB422D+3490733]
	GetHandleVerifier [0x00007FF712DCBA13+3586963]
	GetHandleVerifier [0x00007FF712DC144D+3544525]
	GetHandleVerifier [0x00007FF712B2C9AA+838442]
	(No symbol) [0x00007FF7129DD01F]
	(No symbol) [0x00007FF7129D95E4]
	(No symbol) [0x00007FF7129D9786]
	(No symbol) [0x00007FF7129C8CB9]
	BaseThreadInitThunk [0x00007FFE5459257D+29]
	RtlUserThreadStart [0x00007FFE5596AF08+40]
